In [1]:
## P1 - Parte Computacional - PSI3471
## Aluno: Leonardo Isao Komura - NUSP: 11261656

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from tensorflow import keras

In [3]:
# Leitura dos arquivos
ecg_treino = pd.read_csv("ecg_treino.csv")
rotulos_treino = pd.read_csv("rotulos_treino.csv")
n_rotulos_treino = pd.read_csv("n_rotulos_treino.csv")  

ecg_teste = pd.read_csv("ecg_teste.csv")
rotulos_teste = pd.read_csv("rotulos_teste.csv")
n_rotulos_teste = pd.read_csv("n_rotulos_teste.csv") 

In [85]:
# Transformação em formato numpy
treino = ecg_treino.to_numpy()
r_treino = rotulos_treino.to_numpy()
n_treino = n_rotulos_treino.to_numpy()

teste = ecg_teste.to_numpy()
r_teste = rotulos_teste.to_numpy()
n_teste = n_rotulos_teste.to_numpy()

In [93]:
# Deletando os "nomes" dos pacientes
treino = np.delete(treino, 0, axis=1)
r_treino = np.delete(r_treino, 0, axis=1)
n_treino = np.delete(n_treino, 0, axis=1)

teste = np.delete(teste, 0, axis=1)
r_teste = np.delete(r_teste, 0, axis=1)
n_teste = np.delete(n_teste, 0, axis=1)

print(treino)

[[-0.345 -0.345 -0.345 ... -0.295 -0.29 0.0]
 [-0.31 -0.31 -0.31 ... 0.635 0.265 0.0]
 [-0.125 -0.125 -0.125 ... -0.01 0.04 0.0]
 ...
 [-0.69 -0.69 -0.69 ... -0.16 -0.155 0.0]
 [-0.93 -0.93 -0.93 ... -0.55 -0.555 0.0]
 [-0.38 -0.38 -0.38 ... -0.59 -0.59 0.0]]


In [94]:
# Cálculo da média do "comprimento" dos 50 primeiros batimentos de todos pacientes
nn = 0.0
for i in range(0, len(n_treino)):
    for j in range(50):
        nn = nn + abs(n_treino[i,j] - n_treino[i, j+1])
nn_average = math.ceil(nn/(50*len(n_treino)))
print("Média do comprimento dos 50 primeiros batimentos de todos pacientes: ", nn_average)

Média do comprimento dos 50 primeiros batimentos de todos pacientes:  293


In [97]:
# Colocando cada batimento numa posição de um tensor
n_pontos = 650000
batimentos = int(n_pontos/nn_average)
tensor = np.ndarray(shape=(len(treino), batimentos, nn_average-1), dtype=float)
for i in range(len(treino)):
    for j in range(batimentos):
        tensor[i,j] = treino[i, j*nn_average : (j+1)*nn_average-1]

In [99]:
# Transformada de Fourier da amostra
for i in range(len(treino)):
    for j in range(batimentos):
        tensor[i,j] = np.fft.fft(tensor[i,j])

C:\Users\leona\AppData\Local\Temp\ipykernel_32944\1469831185.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  tensor[i,j] = np.fft.fft(tensor[i,j])


In [9]:
# Cálculo da média do "comprimento" dos 5 primeiros batimentos de cada paciente e adicionando na primeira coluna
for i in range(0, len(n_treino)):
    nn = 0.0
    for j in range(5):
        nn = nn + abs(n_treino[i,j] - n_treino[i, j+1])
    nn_average = math.ceil(nn/5)
    #ft_treino[i,0] = nn_average
    
for i in range(0, len(n_teste)):
    nn = 0.0
    for j in range(5):
        nn = nn + abs(n_teste[i,j] - n_teste[i, j+1])
    nn_average = math.ceil(nn/5)
    #ft_teste[i,0] = nn_average

In [10]:
## Retornando para dataframe
#df_treino = pd.DataFrame(abs(ft_treino))
#df_teste  = pd.DataFrame(abs(ft_teste))
#print(df_treino)

     0         1           2           3           4           5     \
0   326.0   932.270   81.729275   16.953190   71.655701   37.107040   
1   370.0   458.445   64.047180   74.960232   56.200345   48.670853   
2   347.0   888.025   48.018409   34.903937   37.268001   31.381784   
3   236.0  1319.685  336.154171  182.599576   51.806113    7.201617   
4   254.0  2725.010   95.813548   59.146026   35.257334   24.144408   
5   410.0   586.790   39.299156   23.291510   18.722261   21.581286   
6   364.0  1464.690   61.929660   58.126685   41.894234   30.967210   
7   272.0  2969.160  117.537617   18.449536  126.646190   86.901135   
8   304.0  2854.820  100.158811   25.371796   51.955578    9.735718   
9   333.0  2565.570   47.733328   76.590257  234.550956  109.994679   
10  232.0  2383.620   93.940405   37.907058   41.467538   29.387012   
11  441.0  2164.985  223.473675   80.878745   30.939598   52.954324   
12  253.0   535.370   75.901854   42.519269   34.308745   36.043972   
13  23

In [11]:
class MLP(nn.Module):
    def __init__(self, input_size, n_hidden, output_size):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.network = nn.Sequential(
            nn.Linear(input_size, n_hidden), 
            nn.ELU(),
            nn.Linear(n_hidden, n_hidden-10), 
            nn.ELU(),
            nn.Linear(n_hidden-10, n_hidden-10), 
            nn.ELU(),
            nn.Linear(n_hidden-10, output_size), 
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = x.view(-1, self.input_size)
        return self.network(x)

AttributeError: 'float' object has no attribute 'Module'

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def train(epoch, model):
    # Coloca o modelo em modo de treinamento
    model.train()
    
    # Loop sobre os mini-batches, fornecidos pelo DataLoader train_loader
    for batch_idx, (data, target) in enumerate(ft_teste):      
        # Para mandar os dados para o device (GPU ou CPU definido anteriormente), usamos o método .to(device)
        data, target = data.to(device), target.to(device)
        
        # Ajuste de dimensões
        data = data.view(-1, 1, 28, 28)

        # Necessário no PyTorch, para limpar o cache de gradientes acumulados
        optimizer.zero_grad()
        
        # Cálculo da saída
        output = model(data)
        
        # nll_loss é a função custo da entropia cruzada
        loss = F.cross_entropy(output, target)
        
        # cálculo dos gradientes
        loss.backward()
        
        # atualização dos parâmetros do modelo
        optimizer.step()
        
        # Exibe o status do treinamento
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model):
    # Coloca o modelo em modo de teste
    model.eval()
    
    # Variáveis usadas para contabilizar o valor da função custo e número de acertos
    test_loss = 0
    correct = 0
    
    # Loop sobre os mini-batches, fornecidos pelo DataLoader test_loader
    for data, target in ft_teste:
        
        # Para mandar os dados para o device (GPU ou CPU definido anteriormente), usamos o método .to(device)     
        data, target = data.to(device), target.to(device)
        
        # Ajuste de dimensões
        data = data.view(-1, 1, 28, 28)
        
        # Cálculo da saída
        output = model(data)

        # Valor da função custo
        test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss                                                               

        # Cálculo do número de acertos
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    # Mostra o desempenho obtido no teste    
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    accuracy_list.append(accuracy)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))